# DeepDream Tricks

This notebook explores advanced techniques for DeepDream image generation, focusing on methods that enhance the quality, stability, and creativity of the results. We'll investigate several key elements that make DeepDream more effective:

1. **Gradient Smoothing** - How different smoothing methods reduce artifacts
2. **Image Pyramids** - Multi-scale processing for better coherence
3. **Random Shifts** - How jittering prevents grid artifacts
4. **Layer Selection** - Targeting different network layers for varied effects
5. **Parameter Tuning** - Finding optimal settings for your images

By the end of this notebook, you'll understand how these techniques work together to create more compelling DeepDream visualizations and be able to customize the process for your own artistic experiments.

## Setup and Imports

First, we'll import the necessary libraries and set up our environment. We'll need TensorFlow for the neural network operations, various image processing utilities, and our custom DeepDream implementation.

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import torch
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights
import cv2 as cv

# Import our DeepDream implementation and utilities
from deepdreaming import img, utils, smoothing, deepdream as dd
from deepdreaming.utils import read_image_net_classes
from deepdreaming.config import DreamConfig, GradSmoothingMode

# Set random seeds for reproducibility
SEED = 42


def set_seeds(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seeds(SEED)

# Configure matplotlib for inline display
%matplotlib inline
%load_ext autoreload
%autoreload 2

Now that we've imported the necessary libraries, let's set up the paths to our data directories and model configuration.

In [ ]:
# Configuration paths
TARGET_SHAPE = (224, 224, 3)  # Standard input size for VGG16
IMAGE_NET_CLASSES_PATH = "input/image_net_classes/imagenet_classes.txt"
INPUT_IMAGES_DIR = "input/deepdream-sample-images"
OUTPUT_IMAGES_DIR = "output"

# Verify paths exist
assert os.path.exists(IMAGE_NET_CLASSES_PATH), "Please provide a valid `IMAGE_NET_CLASSES_PATH`"
assert os.path.exists(INPUT_IMAGES_DIR), "Please provide a valid `INPUT_IMAGES_DIR`"
assert os.path.exists(OUTPUT_IMAGES_DIR), "Please provide a valid `DEEPDREAM_OUTPUT_PATH`"

# Set up model
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_TYPE = vgg16
WEIGHTS = VGG16_Weights.DEFAULT

# Load pre-trained model
MODEL = MODEL_TYPE(weights=WEIGHTS).to(DEVICE).eval()
MODEL.requires_grad_(False)
IMAGE_NET_CLASSES = read_image_net_classes(IMAGE_NET_CLASSES_PATH)

print(f"Model loaded on {DEVICE}")
print(f"Input images will be loaded from {INPUT_IMAGES_DIR}")
print(f"Output images will be saved to {OUTPUT_IMAGES_DIR}")

## 1. Basic DeepDream Implementation

Before diving into advanced techniques, let's establish a baseline with a simple DeepDream implementation. 
This will help us appreciate how each advanced technique enhances our results.

### What is DeepDream?

DeepDream is a computer vision technique created by Alexander Mordvintsev at Google. It uses neural networks to find and enhance patterns in images, creating dream-like hallucinogenic appearances. The algorithm works by:

1. Taking a trained convolutional neural network
2. Selecting a layer in the network
3. Setting an optimization objective to maximize the activations of that layer
4. Iteratively modifying the input image to increase these activations

Let's implement a basic version first:

In [ ]:
# Load a single test image
test_image_path = os.path.join(INPUT_IMAGES_DIR, "tree.png")  # Change to an image in your directory
test_image = img.io.read_image(test_image_path, TARGET_SHAPE)

# Create a basic DeepDream configuration
basic_config = DreamConfig(
    pyramid_layers=1,  # No multi-scale processing yet
    num_iter=10,
    learning_rate=0.08,
    grad_smoothing=GradSmoothingMode.Disable,  # No smoothing yet
    shift_size=0,  # No random shifts yet
)

# Initialize DeepDream targeting middle layers of VGG16
# These layers typically capture interesting features without being too abstract
deepdream = dd.DeepDream(MODEL, ["features[22]"])  # Layer 22 is a good starting point for VGG16

# Generate a basic dream
basic_dream = deepdream.dream(test_image, config=basic_config)

# Display the result
utils.display_two_img(test_image, basic_dream, figsize=(16, 8))
plt.show()

### Understanding the Results

In the basic implementation above:
- We used a single scale (no image pyramid)
- No gradient smoothing was applied
- No random shifts were used
- We only targeted a single layer (features[22])

Notice that the results likely show some patterns but may have artifacts, grid-like patterns, or look somewhat "harsh." This is because we haven't applied any of the advanced techniques that make DeepDream more visually appealing.

Next, let's explore these advanced techniques one by one.

## 2. Image Pyramids: Multi-Scale Processing

One of the most powerful techniques for improving DeepDream results is using image pyramids. This technique processes the image at multiple scales, starting from small and working up to full size.

### Why use image pyramids?

1. **Coherent structure**: Creates more coherent, large-scale patterns
2. **Detail across scales**: Combines details from multiple resolutions
3. **Better convergence**: Prevents getting stuck in local optima

Let's compare our basic dream with one that uses an image pyramid:

In [ ]:
# Import the ImagePyramid class to better understand how it works
from deepdreaming.pyramid import ImagePyramid

# Create an image pyramid to visualize the scales
pyramid_demo = ImagePyramid(test_image.shape, layers=5, ratio=0.7)

# Display the scales that will be used in the pyramid
scales = list(pyramid_demo)  # Get all scales from the pyramid iterator
print(f"Original image shape: {test_image.shape[:2]}")
print("Pyramid scales (from smallest to largest):")
for i, scale in enumerate(scales):
    print(f"  Layer {i+1}: {scale}")

# Visualize the pyramid scales
fig, axes = plt.subplots(1, len(scales), figsize=(15, 15))
for i, scale in enumerate(scales):
    # Resize image to this scale
    resized = cv.resize(test_image, (scale[1], scale[0]))
    axes[i].imshow(resized)
    axes[i].set_title(f"Layer {i+1}: {scale}")
    axes[i].axis("off")
plt.tight_layout()
plt.show()

### Try Filtering After Each Pyramid Layer

Curious to experiment? You can apply a filter (like Gaussian smoothing) right after each upsampling step in the image pyramid. This isn't implemented here, but adding such filtering could give your DeepDreams a different, possibly smoother look. Give it a shot and see what new effects you discover!


Now that we understand how image pyramids work, let's apply this technique to our DeepDream process.

In [ ]:
# Create a configuration with image pyramid enabled
pyramid_config = DreamConfig(
    pyramid_layers=5,  # Process at 5 different scales
    pyramid_ratio=0.7,  # Each layer is 70% the size of the previous
    num_iter=10,
    learning_rate=0.08,
    grad_smoothing=GradSmoothingMode.Disable,  # Still no smoothing
    shift_size=0,  # Still no random shifts
)

# Use the same layer as before
deepdream = dd.DeepDream(MODEL, ["features[22]"])

# Generate a dream with image pyramid
pyramid_dream = deepdream.dream(test_image, config=pyramid_config)

# Compare the results
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
axes[0].imshow(test_image)
axes[0].set_title("Original Image")
axes[0].axis("off")

axes[1].imshow(basic_dream)
axes[1].set_title("Basic Dream (No Pyramid)")
axes[1].axis("off")

axes[2].imshow(pyramid_dream)
axes[2].set_title("With Image Pyramid")
axes[2].axis("off")

plt.tight_layout()
plt.show()

### Image Pyramid Effect Analysis

The pyramid-enhanced dream should show more coherent and large-scale patterns. You'll notice:

1. The dream patterns follow the structure of the original image better
2. The features appear at multiple scales (both large and small details)
3. The overall effect tends to be more aesthetically pleasing

Now let's explore how changing pyramid parameters affects results:

In [ ]:
# Let's try different pyramid configurations
pyramid_configs = [
    (
        "Few Large Steps",
        DreamConfig(
            pyramid_layers=3,
            pyramid_ratio=0.5,
            num_iter=10,
            grad_smoothing=GradSmoothingMode.Disable,
            gradient_norm=False,
        ),
    ),
    (
        "Many Small Steps",
        DreamConfig(
            pyramid_layers=8,
            pyramid_ratio=0.85,
            num_iter=10,
            grad_smoothing=GradSmoothingMode.Disable,
            gradient_norm=False,
        ),
    ),
]

results = []
for name, config in pyramid_configs:
    result = deepdream.dream(test_image, config=config)
    results.append((name, result))

# Display the results
fig, axes = plt.subplots(1, len(results) + 1, figsize=(18, 6))
axes[0].imshow(test_image)
axes[0].set_title("Original Image")
axes[0].axis("off")

for i, (name, result) in enumerate(results):
    axes[i + 1].imshow(result)
    axes[i + 1].set_title(f"Pyramid: {name}")
    axes[i + 1].axis("off")

plt.tight_layout()
plt.show()

## 3. Gradient Smoothing Techniques

Another important technique for improving DeepDream is gradient smoothing. This helps reduce high-frequency artifacts and creates more natural-looking results.

Our implementation supports two types of gradient smoothing:
1. **Box Smoothing**: Simple averaging of neighboring pixels
2. **Gaussian Smoothing**: Weighted averaging where closer pixels have more influence

Let's see how these smoothing techniques affect our results:

In [ ]:
# Read the noisy image
noisy_image = img.io.read_image(os.path.join(INPUT_IMAGES_DIR, "noisy_image.jpg"))

# Convert to tensor for smoothing functions
noisy_tensor = img.proc.to_tensor(noisy_image).float()

# Apply different smoothing techniques
no_smooth = noisy_tensor.clone()
box_smooth = smoothing.box_smoothing(noisy_tensor, kernel_size=9, padding_mode="reflect")
gauss_smooth1 = smoothing.gaussian_smoothing(noisy_tensor, kernel_size=9, padding_mode="reflect", sigma=(2,))
gauss_smooth2 = smoothing.gaussian_smoothing(noisy_tensor, kernel_size=9, padding_mode="reflect", sigma=(4,))
gauss_blend = smoothing.gaussian_smoothing(
    noisy_tensor, kernel_size=9, padding_mode="reflect", sigma=(2, 4), blending_weights=(0.7, 0.3)
)

# Convert back to numpy for visualization
images = [
    ("Original", img.proc.to_image(no_smooth)),
    ("Box Smoothing", img.proc.to_image(box_smooth)),
    ("Gaussian (σ=2)", img.proc.to_image(gauss_smooth1)),
    ("Gaussian (σ=4)", img.proc.to_image(gauss_smooth2)),
    ("Blended Gaussian", img.proc.to_image(gauss_blend)),  # (σ_1=2, σ_2=4) (w_1=0.7, w_2=0.3)
]

# Display the results
fig, axes = plt.subplots(1, len(images), figsize=(18, 4))
for i, (name, image) in enumerate(images):
    axes[i].imshow(image)
    axes[i].set_title(name)
    axes[i].axis("off")
plt.tight_layout()
plt.show()

Now that we understand how smoothing works, let's apply these techniques to our DeepDream process to see how they affect the final result.

In [ ]:
# Create configs with different smoothing settings
smoothing_configs = [
    ("No Smoothing", DreamConfig(pyramid_layers=7, pyramid_ratio=0.75, grad_smoothing=GradSmoothingMode.Disable)),
    (
        "Box Smoothing",
        DreamConfig(
            pyramid_layers=7,
            pyramid_ratio=0.75,
            grad_smoothing=GradSmoothingMode.BoxSmoothing,
            grad_smoothing_kernel_size=3,
        ),
    ),
    (
        "Gaussian Smoothing",
        DreamConfig(
            pyramid_layers=7,
            pyramid_ratio=0.75,
            grad_smoothing=GradSmoothingMode.GaussianSmoothing,
            grad_smoothing_kernel_size=3,
            grad_smoothing_gaussian_sigmas=(0.5,),
            grad_smoothing_gaussian_blending_weights=(1.0,),
        ),
    ),
    (
        "Multi-scale Gaussian",
        DreamConfig(
            pyramid_layers=7,
            pyramid_ratio=0.75,
            grad_smoothing=GradSmoothingMode.GaussianSmoothing,
            grad_smoothing_kernel_size=5,
            grad_smoothing_gaussian_sigmas=(0.5, 1.5, 2.5),
            grad_smoothing_gaussian_blending_weights=(0.6, 0.3, 0.1),
        ),
    ),
]

# Generate dreams with different smoothing settings
smoothing_results = []
for name, config in smoothing_configs:
    result = deepdream.dream(test_image, config=config)
    smoothing_results.append((name, result))

# Display the results in a grid
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for i, (name, result) in enumerate(smoothing_results):
    axes[i].imshow(result)
    axes[i].set_title(name)
    axes[i].axis("off")

plt.tight_layout()
plt.show()

### Gradient Smoothing Effect Analysis

Looking at the results, you can observe:

1. **No Smoothing**: Often shows grid-like artifacts and sharper but noisier patterns
2. **Box Smoothing**: Reduces artifacts but may blur details uniformly
3. **Gaussian Smoothing**: Preserves more structure while reducing noise
4. **Multi-scale Gaussian**: Balances detail preservation and artifact removal

Gaussian smoothing tends to produce the most natural results, especially when using multiple scales with appropriate blending weights.

## 4. Random Shifts: Preventing Grid Artifacts

Another technique to improve DeepDream results is random shifts (also called jittering). This technique:

1. Randomly shifts the image by a few pixels before each optimization step
2. Shifts it back after the gradient update
3. Helps prevent grid artifacts that can emerge from the convolutional structure of the network

Let's see how random shifts affect our results:

In [ ]:
# Import the RandomShift class to understand how it works
from deepdreaming.shift import RandomShift

# Use a real image to better visualize the effect of random shifts
pattern_image = img.io.read_image("assets/tree.png", TARGET_SHAPE)
pattern_tensor = img.proc.to_tensor(pattern_image)

# Apply a random shift
shift_demo = RandomShift(shift_size=128)
shifted = shift_demo.shift(pattern_tensor)
shifted_back = shift_demo.shift_back(shifted)

# Display the shift effect
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
axes[0].imshow(img.proc.to_image(pattern_tensor))
axes[0].set_title("Original Image")
axes[0].axis("off")

axes[1].imshow(img.proc.to_image(shifted))
axes[1].set_title(f"Shifted ({shift_demo.horizontal}, {shift_demo.vertical})")
axes[1].axis("off")

axes[2].imshow(img.proc.to_image(shifted_back))
axes[2].set_title("Shifted Back")
axes[2].axis("off")

plt.tight_layout()
plt.show()

Now let's apply random shifts to our DeepDream process:

In [ ]:
# Create configs with and without random shifts
shift_configs = [
    (
        "No Shifts",
        DreamConfig(
            pyramid_layers=7,
            pyramid_ratio=0.75,
            grad_smoothing=GradSmoothingMode.GaussianSmoothing,
            shift_size=0,  # Disable random shifts
        ),
    ),
    (
        "Small Shifts",
        DreamConfig(
            pyramid_layers=7,
            pyramid_ratio=0.75,
            grad_smoothing=GradSmoothingMode.GaussianSmoothing,
            shift_size=8,  # Small random shifts
        ),
    ),
    (
        "Medium Shifts",
        DreamConfig(
            pyramid_layers=7,
            pyramid_ratio=0.75,
            grad_smoothing=GradSmoothingMode.GaussianSmoothing,
            shift_size=32,  # Medium random shifts
        ),
    ),
    (
        "Large Shifts",
        DreamConfig(
            pyramid_layers=7,
            pyramid_ratio=0.75,
            grad_smoothing=GradSmoothingMode.GaussianSmoothing,
            shift_size=64,  # Large random shifts
        ),
    ),
]

# Generate dreams with different shift settings
shift_results = []
for name, config in shift_configs:
    result = deepdream.dream(test_image, config=config)
    shift_results.append((name, result))

# Display the results in a grid
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for i, (name, result) in enumerate(shift_results):
    axes[i].imshow(result)
    axes[i].set_title(name)
    axes[i].axis("off")

plt.tight_layout()
plt.show()

### Random Shifts Effect Analysis

Looking at the results, you may notice:

1. **No Shifts**: May show regular grid-like patterns or tiling artifacts
2. **Small Shifts**: Reduces minor artifacts while maintaining detail
3. **Medium Shifts**: Further reduces artifacts but may slightly blur some details
4. **Large Shifts**: Most effective at preventing artifacts but may lose some fine detail

The optimal shift size often depends on the image resolution and the specific patterns you want to create.

## 5. Layer Selection: Targeting Different Network Layers

The choice of which network layers to target has a dramatic effect on the resulting dream patterns. Different layers in the network learn to recognize different types of features:

- **Early layers**: Detect simple features like edges, colors, and textures
- **Middle layers**: Detect parts of objects like wheels, windows, or fur
- **Deep layers**: Detect whole objects and scenes

Let's explore how targeting different layers affects our DeepDream results:

In [ ]:
# Define layer groups for VGG16
layer_groups = [
    ("Early Layers", ["features[3]", "features[8]"]),  # Detect edges and textures
    ("Middle Layers", ["features[15]", "features[22]"]),  # Detect parts and patterns
    ("Deep Layers", ["features[29]"]),  # Detect objects
    ("Mixed Layers", ["features[8]", "features[15]", "features[29]"]),  # Mixed effects
]

# Create a standard config with all our techniques enabled
standard_config = DreamConfig(
    pyramid_layers=4,
    pyramid_ratio=0.7,
    num_iter=12,
    learning_rate=0.09,
    grad_smoothing=GradSmoothingMode.GaussianSmoothing,
    grad_smoothing_kernel_size=3,
    grad_smoothing_gaussian_sigmas=(0.5, 1.5),
    grad_smoothing_gaussian_blending_weights=(0.6, 0.4),
    shift_size=32,
)

# Generate dreams targeting different layers
layer_results = []
for name, layers in layer_groups:
    # Create a new DeepDream instance targeting these layers
    dream_instance = dd.DeepDream(MODEL, layers)
    result = dream_instance.dream(test_image, config=standard_config)
    layer_results.append((name, layers, result))

# Display the results
fig, axes = plt.subplots(len(layer_groups), 2, figsize=(12, 4 * len(layer_groups)))

for i, (name, layers, result) in enumerate(layer_results):
    # Display the original image on the left
    if i == 0:
        axes[i, 0].imshow(test_image)
        axes[i, 0].set_title("Original Image")
    else:
        axes[i, 0].imshow(test_image)
        axes[i, 0].set_title("Original Image")

    # Display the dream result on the right
    axes[i, 1].imshow(result)
    axes[i, 1].set_title(f"{name}: {', '.join(layers)}")

    # Turn off axis ticks
    axes[i, 0].axis("off")
    axes[i, 1].axis("off")

plt.tight_layout()
plt.show()

### Layer Selection Analysis

From the results, you can observe how different layers create different visual effects:

1. **Early Layers**: Produce more texture and color-based transformations
2. **Middle Layers**: Create more recognizable patterns and object parts
3. **Deep Layers**: May generate whole objects or more abstract patterns
4. **Mixed Layers**: Combine effects from multiple levels of abstraction

The choice of which layers to target depends on your artistic goals:
- Use early layers for subtle texture enhancement
- Use middle layers for recognizable but surreal patterns
- Use deep layers for more hallucinogenic transformations
- Mix layers for complex effects

## 6. Putting It All Together: Parameter Exploration

Now that we understand the key techniques, let's explore how different combinations of parameters affect our results. We'll create a comprehensive parameter grid to showcase various effects.

In [ ]:
# Choose a different test image for variety
test_image2_path = os.path.join(INPUT_IMAGES_DIR, "messi.jpg")  # Change to an image in your directory
test_image2 = img.io.read_image(test_image2_path, TARGET_SHAPE)

# Create a parameter grid
parameter_configs = [
    ("Default Settings", DreamConfig()),
    (
        "Subtle Effect",
        DreamConfig(
            pyramid_layers=3,
            pyramid_ratio=0.8,
            num_iter=5,
            learning_rate=0.05,
            grad_smoothing=GradSmoothingMode.GaussianSmoothing,
            grad_smoothing_gaussian_sigmas=(0.5,),
            shift_size=16,
        ),
    ),
    (
        "Intense Effect",
        DreamConfig(
            pyramid_layers=6,
            pyramid_ratio=0.75,
            num_iter=20,
            learning_rate=0.12,
            grad_smoothing=GradSmoothingMode.GaussianSmoothing,
            grad_smoothing_gaussian_sigmas=(1.0, 2.0),
            grad_smoothing_gaussian_blending_weights=(0.7, 0.3),
            shift_size=48,
        ),
    ),
    (
        "Surreal Effect",
        DreamConfig(
            pyramid_layers=4,
            pyramid_ratio=0.7,
            num_iter=15,
            learning_rate=0.1,
            grad_smoothing=GradSmoothingMode.BoxSmoothing,
            grad_smoothing_kernel_size=5,
            shift_size=64,
        ),
    ),
]

# Create DeepDream instances with different layer targets
dream_early = dd.DeepDream(MODEL, ["features[8]"])
dream_middle = dd.DeepDream(MODEL, ["features[15]", "features[22]"])
dream_deep = dd.DeepDream(MODEL, ["features[29]"])

# Generate and display a comprehensive grid of results
rows = len(parameter_configs)
cols = 3  # Three layer configurations

fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))

for i, (name, config) in enumerate(parameter_configs):
    # Generate dreams with different layer targets but same parameter config
    result_early = dream_early.dream(test_image2, config=config)
    result_middle = dream_middle.dream(test_image2, config=config)
    result_deep = dream_deep.dream(test_image2, config=config)

    # Display the results in a row
    axes[i, 0].imshow(result_early)
    axes[i, 0].set_title(f"{name}\nEarly Layers")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(result_middle)
    axes[i, 1].set_title(f"{name}\nMiddle Layers")
    axes[i, 1].axis("off")

    axes[i, 2].imshow(result_deep)
    axes[i, 2].set_title(f"{name}\nDeep Layers")
    axes[i, 2].axis("off")

plt.tight_layout()
plt.show()

## 7. Practical Applications & Image Gallery

Let's apply our knowledge to create a gallery of interesting DeepDream images using different source images and configurations. This will demonstrate the versatility and creative potential of the techniques we've learned.

In [ ]:
# Choose a set of diverse images
image_files = ["vader.jpeg", "yoda.jpeg", "bombordiro_crocodilo.jpeg", "tralalelo_tralala.jpeg"]
images = []

for file in image_files:
    file_path = os.path.join(INPUT_IMAGES_DIR, file)
    if os.path.exists(file_path):
        image = img.io.read_image(file_path, TARGET_SHAPE)
        images.append((file, image))

# Create some creative configurations
creative_configs = [
    (
        "Dreamy",
        dd.DeepDream(MODEL, ["features[22]"]),
        DreamConfig(
            pyramid_layers=4,
            pyramid_ratio=0.8,
            num_iter=12,
            learning_rate=0.09,
            grad_smoothing=GradSmoothingMode.GaussianSmoothing,
            grad_smoothing_kernel_size=3,
            grad_smoothing_gaussian_sigmas=(0.5, 1.0),
            grad_smoothing_gaussian_blending_weights=(0.7, 0.3),
            shift_size=32,
        ),
    ),
    (
        "Psychedelic",
        dd.DeepDream(MODEL, ["features[8]", "features[15]", "features[29]"]),
        DreamConfig(
            pyramid_layers=5,
            pyramid_ratio=0.7,
            num_iter=15,
            learning_rate=0.1,
            grad_smoothing=GradSmoothingMode.GaussianSmoothing,
            grad_smoothing_kernel_size=5,
            grad_smoothing_gaussian_sigmas=(0.5, 1.5, 2.5),
            grad_smoothing_gaussian_blending_weights=(0.5, 0.3, 0.2),
            shift_size=48,
        ),
    ),
    (
        "Abstract",
        dd.DeepDream(MODEL, ["features[3]", "features[29]"]),
        DreamConfig(
            pyramid_layers=6,
            pyramid_ratio=0.75,
            num_iter=20,
            learning_rate=0.12,
            grad_smoothing=GradSmoothingMode.Disable,
            grad_smoothing_kernel_size=7,
            shift_size=64,
        ),
    ),
]

# Process each image with each creative configuration
for file_name, image in images:
    # Create a figure for this image and its transformations
    fig, axes = plt.subplots(1, len(creative_configs) + 1, figsize=(18, 5))

    # Display original
    axes[0].imshow(image)
    axes[0].set_title(f"Original: {file_name}")
    axes[0].axis("off")

    # Process with each creative configuration
    for i, (name, dream_instance, config) in enumerate(creative_configs):
        # Generate the dream
        result = dream_instance.dream(image, config=config)

        # Display the result
        axes[i + 1].imshow(result)
        axes[i + 1].set_title(f"{name} Effect")
        axes[i + 1].axis("off")

        # Save the result if desired
        output_path = os.path.join(OUTPUT_IMAGES_DIR, f"{file_name.split('.')[0]}_{name}.jpg")
        cv.imwrite(output_path, img.proc.to_cv(result))

    plt.tight_layout()
    plt.show()

## 8. Conclusion and Next Steps

In this notebook, we've explored several advanced techniques for creating better DeepDream visualizations:

1. **Image Pyramids** for multi-scale processing
2. **Gradient Smoothing** to reduce artifacts
3. **Random Shifts** to prevent grid patterns
4. **Layer Selection** for different visual effects
5. **Parameter Tuning** for customizing the dream style

Each of these techniques contributes to creating more visually appealing and coherent dream images. By combining them effectively, you can create a wide range of artistic effects.

### Next Steps

To take your DeepDream explorations further, you might want to try:

1. Using different pre-trained models (ResNet, Inception, etc.)
2. Animating DeepDream by creating frame sequences
3. Exploring guided dreams (feature transfer) in our companion notebook
4. Applying DeepDream to specific regions of an image
5. Creating hybrid dreams by blending multiple techniques

Check out the "DeepDream Guided" notebook for more advanced techniques focused on feature transfer between images.